In [1]:
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np


from sklearn.preprocessing import StandardScaler 
# from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
# from sklearn.externals import joblib

import lightgbm as lgb

import gc


# fix random seed for reproducibility
seed = 0
np.random.seed(seed)

In [3]:
# import processed_data
data_dir = "/home/lee/Documents/DatasetsForGitHub/Taxi_Trajectory_Prediction/"
df_train = pd.read_pickle(data_dir+"train.pkl")
df_test = pd.read_pickle(data_dir+"test.pkl")

In [9]:
#create X, Y for model input
X_train = df_train.drop(['gap_forecast'], axis = 1)
y_train = df_train['gap_forecast']

X_test = df_test.drop(['gap_forecast'], axis = 1)
y_test = df_test['gap_forecast']

#scale data
scaler = StandardScaler()
scaler.fit(X_train)  

#transform data based on scaler
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

In [11]:
X_train.shape

(16773120, 19)

In [10]:
reg_lgb = lgb.LGBMRegressor(random_state=0)
reg_lgb.fit(X_train, y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
       n_jobs=-1, num_leaves=31, objective=None, random_state=0,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)

In [20]:
#determine accuracy

print(mean_squared_error(y_train, reg_lgb.predict(X_train)))
print(mean_squared_error(y_test, reg_lgb.predict(X_test)))

0.1661852924035559
0.15680317537677682


In [22]:
import pandas as pd
import seaborn as sns
import matplotlib.image as image
import matplotlib.pylab as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:

results = pd.concat([df_test, reg_lgb.predict(X_test)], axis=1)
results['FRAME'] = results['week'].astype(str) + "-" + results['dayofweek'].astype(str) + "-" \
                    + results['time_window'].astype(str)

#split result based on timefram based on the timesteps
g = results.drop(['week', 'dayofweek', 'time_window'], axis=1)
g['FRAME'] = LabelEncoder().fit_transform(g.FRAME)

print 'create frame output...'
#create frame output
for t in range(600, 888):
    r = g[g.FRAME == t]
    
    FC = r.groupby(by=['X', 'Y'], as_index=False).GAP_FCST.sum()
    PRED = r.groupby(by=['X', 'Y'], as_index=False).GAP_PRED.sum()
    
    im = image.imread('porto.png')
    fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True, figsize=(16,5))
    ax1.set_title("PREDICTED GAP")
    ax2.set_title("ACTUAL GAP")

    ax1.imshow(im, extent=[0,10,0,7], aspect='auto')
    ax2.imshow(im, extent=[0,10,0,7], aspect='auto')
    
    FCS = FC.pivot('Y', 'X', 'GAP_FCST')
    PREDS = PRED.pivot('Y', 'X', 'GAP_PRED')
       
    f1=sns.heatmap(PREDS, ax=ax1, vmin=-10, vmax=10, cmap='seismic', alpha=0.6, robust=True)
    f2=sns.heatmap(FCS, ax=ax2, vmin=-10, vmax=10, cmap='seismic', alpha=0.6)
    
    f1.invert_yaxis()
    
    fig.savefig("output/file_" + str(t) + ".png")
    fig.clf()
    plt.close(fig)

In [ ]:
# # define the model
# def larger_model():
#     # create model
#     model = Sequential()
#     model.add(Dense(32, input_dim=19, kernel_initializer='normal', activation='relu'))
#     model.add(Dense(8, kernel_initializer='normal', activation='relu'))
#     model.add(Dense(1, kernel_initializer='normal'))
#     # Compile model
#     model.compile(loss='mean_squared_error', optimizer='adam')
#     return model


# # fix random seed for reproducibility
# seed = 0
# np.random.seed(seed)

# epochs = 32
# batch_size = 5
    
# # evaluate model with standardized dataset
# estimator = KerasRegressor(build_fn=larger_model, epochs=epochs, batch_size=batch_size, verbose=1)

# kfold = KFold(n_splits=5, random_state=seed)
# results = cross_val_score(estimator, X_train, y_train, cv=kfold)